# ML3_CNN_MNIST - Tania_El_Achkar

In [17]:
'''
A Convolutional Network implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)
'''

'\nA Convolutional Network implementation example using TensorFlow library.\nThis example is using the MNIST database of handwritten digits\n(http://yann.lecun.com/exdb/mnist/)\n'

In [18]:
from __future__ import print_function
import tensorflow as tf

In [19]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data

In [20]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [21]:
# Parameters
learning_rate = 0.001
training_iters = 200000
batch_size = 128
display_step = 10

In [22]:
# Network Parameters
n_input = 784   # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)
dropout = 0.75  # Dropout, probability to keep units

In [23]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)  # dropout (keep probability)

In [24]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

In [25]:
# Create model
def conv_net(x, weights, biases, dropout):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    print("shape of conv1: ", conv1.get_shape())

    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)
    print("shape of conv1 after max pool: ", conv1.get_shape())

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    print("shape of conv2: ", conv2.get_shape())

    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    print("shape of conv1 after max pool: ", conv2.get_shape())
    exit(-1)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [26]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([7 * 7 * 64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [27]:
# Construct model
pred = conv_net(x, weights, biases, keep_prob)

shape of conv1:  (?, 28, 28, 32)
shape of conv1 after max pool:  (?, 14, 14, 32)
shape of conv2:  (?, 14, 14, 64)
shape of conv1 after max pool:  (?, 7, 7, 64)


In [28]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [29]:
# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [30]:
# Initializing the variables
init = tf.global_variables_initializer()

In [31]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    step = 1
    # Keep training until reach max iterations
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y,
                                       keep_prob: dropout})
        if step % display_step == 0:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print("Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
                  "{:.6f}".format(loss) + ", Training Accuracy= " + \
                  "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")

    # Calculate accuracy for 256 mnist test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: mnist.test.images[:256],
                                      y: mnist.test.labels[:256],
                                      keep_prob: 1.}))

Iter 1280, Minibatch Loss= 20855.632812, Training Accuracy= 0.32812
Iter 2560, Minibatch Loss= 11116.433594, Training Accuracy= 0.52344
Iter 3840, Minibatch Loss= 5729.418945, Training Accuracy= 0.72656
Iter 5120, Minibatch Loss= 3601.786133, Training Accuracy= 0.78906
Iter 6400, Minibatch Loss= 3238.329346, Training Accuracy= 0.81250
Iter 7680, Minibatch Loss= 2684.652832, Training Accuracy= 0.78125
Iter 8960, Minibatch Loss= 2639.956055, Training Accuracy= 0.86719
Iter 10240, Minibatch Loss= 3838.337891, Training Accuracy= 0.81250
Iter 11520, Minibatch Loss= 3009.984863, Training Accuracy= 0.85938
Iter 12800, Minibatch Loss= 2604.558105, Training Accuracy= 0.84375
Iter 14080, Minibatch Loss= 1984.638184, Training Accuracy= 0.91406
Iter 15360, Minibatch Loss= 3352.636719, Training Accuracy= 0.85938
Iter 16640, Minibatch Loss= 1330.103027, Training Accuracy= 0.91406
Iter 17920, Minibatch Loss= 1476.274902, Training Accuracy= 0.92188
Iter 19200, Minibatch Loss= 1516.815308, Training Acc